<a href="https://www.kaggle.com/code/jonathankao/dec-2023-tabular-xgboost?scriptVersionId=157045087" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Task Overview 
Task: In this synthetic dataset based off of a real dataset funded by the Mayo Clinic, each example represents both general and survival information about a patient that has liver cirrhosis, a condition involving prolonged liver damage. The goal is to train a machine learning model that can predict the patient's current survival status based on the data features. 

Approach v1.0: Our approach will be to train a Logistic Regression model using sklearn to be used as a baseilne model and to then train a performance-focused model using XGBoost as a learning exercise.

Approach v2.0: After a few days of initial work, we realized that it was feasible within the competition timeframe to improve on Approach 1 by training an Ensemble of models so we changed our approach to training multiple Logistic Regression, Random Forest, and XGBoost models and then combining them into an ensemble.

Next Steps: Add hyperparameter tuning for each model and combine the models together to form an ensemble.  

Version History
1. v1.0-1.3 - Implemented dataset loading and dataset pre-processing. Added explanations and comments for each step. 
2. v1.4 - Added Logistic Regression model training and Kaggle submission formatting
3. v2.0 - Added Random Forest and XGBoost model training and prediction.
4. v2.1 - Added log-loss calculations for each model on the validation set. 
5. v2.2 - Added initial hyperparameter tuning for XGBoost
6. v2.3 - Added LGBM Classifier. Added model consisting of an ensemble of tuned Random Forest, XGBoost, and LGBM models. 

Results Log
1. Initial Logistic Regression Model - ~.52 log-loss score
2. Initial Random Forest Model - ~.45 log-loss score
3. Initial XGBoost Model - ~.42 log-loss score
4. Tuned XGBoost Model ~.404 log-loss score 
5. Tuned LGBM Model - ~.411 log-loss score
6. Ensemble Model - ~.445 log-loss score 

# Dataset Loading

In [1]:
import numpy as np
import pandas as pd

# Read in the training and test data as Pandas DataFrames 
train_file_path = "/kaggle/input/playground-series-s3e26/train.csv"
train_df = pd.read_csv(train_file_path) 
test_file_path = "/kaggle/input/playground-series-s3e26/test.csv"
test_df = pd.read_csv(test_file_path)

# Remove the id column since it is not useful for prediction
test_id_df = test_df['id'].astype(int)
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

# Use the head method to visually see that the dataset has been loaded
train_df.head(5)

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [2]:
# Use info to see the number of categorical columns (any column with Dtype=object is text and will need to be converted to a number) 
# Here you can also see if there are any missing values by looking at the non-null values in each column
train_df.info()

# Make a list of categorical columns for future use (in the feature scaling section)
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         7905 non-null   int64  
 1   Drug           7905 non-null   object 
 2   Age            7905 non-null   int64  
 3   Sex            7905 non-null   object 
 4   Ascites        7905 non-null   object 
 5   Hepatomegaly   7905 non-null   object 
 6   Spiders        7905 non-null   object 
 7   Edema          7905 non-null   object 
 8   Bilirubin      7905 non-null   float64
 9   Cholesterol    7905 non-null   float64
 10  Albumin        7905 non-null   float64
 11  Copper         7905 non-null   float64
 12  Alk_Phos       7905 non-null   float64
 13  SGOT           7905 non-null   float64
 14  Tryglicerides  7905 non-null   float64
 15  Platelets      7905 non-null   float64
 16  Prothrombin    7905 non-null   float64
 17  Stage          7905 non-null   float64
 18  Status  

In [3]:
# Use the describe method to generate statistical information such as standard deviation, mean, and min/max. 
train_df.describe()

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000
mean,2030.173308,18373.146490,2.594485,350.561923,3.548323,83.902846,1816.745250,114.604602,115.340164,265.228969,10.629462,3.032511
std,1094.233744,3679.958739,3.812960,195.379344,0.346171,75.899266,1903.750657,48.790945,52.530402,87.465579,0.781735,0.866511
min,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,1230.000000,15574.000000,0.700000,248.000000,3.350000,39.000000,834.000000,75.950000,84.000000,211.000000,10.000000,2.000000
50%,1831.000000,18713.000000,1.100000,298.000000,3.580000,63.000000,1181.000000,108.500000,104.000000,265.000000,10.600000,3.000000
75%,2689.000000,20684.000000,3.000000,390.000000,3.770000,102.000000,1857.000000,137.950000,139.000000,316.000000,11.000000,4.000000
max,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,563.000000,18.000000,4.000000


# Dataset Pre-Processing Task Overview

1. Target Variable Label Encoding 
2. Missing Feature Value Handling 
3. Feature Scaling 
4. Categorical Attribute Handling (Should be done at the end since it may change the number of columns)
5. Train/Test Split

## Target Variable Label Encoding

Problem: Since the target column, Status, is a text column, we need to convert the column's values into integers so the machine learning model can process it.

Possible Approaches:
1. Label Encoding - Convert each text category into an integer label. 
2. Ordinal Encoding - Convert each text category into an integer label but with a particular order. Used when the categories have some quantitative order that can be taken advantage of like low, medium, high.   
3. One-Hot Encoding - Convert each text category into a separate column. For example, this is done in the softmax layer of a neural network. 

Chosen Approach: Label Encoding - There is no obvious ordering in the 'Status' column and ML libraries typically expect a single target column so we will use label encoding.

In [4]:
# Check to see what the initial text categories are
train_df['Status'].value_counts()

Status
C     4965
D     2665
CL     275
Name: count, dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder

# Use sklearn's LabelEncoder class to transform the Status column's values from strings to integers.
label_encoder = LabelEncoder() 
train_df['Status'] = label_encoder.fit_transform(train_df['Status'])

# Check that the label encoder transformation was applied correctly and the categories are now numbers
train_df['Status'].value_counts()

Status
0    4965
2    2665
1     275
Name: count, dtype: int64

## Missing Feature Value Handling

Problem: We need to check if there are any missing feature values since most machine learning algorithms cannot handle an empty cell. For example, a Logistic Regression model would throw an error in training although other algorithms like XGBoost can train normally by substituting in a value as needed. 

Solution: We check for missing feature values by using the isna DataFrame method which returns a boolean DataFrame where each cell is True if the value is missing and False otherwise. We then apply the sum method to find the number of missing values in each column. 

In [6]:
# Check for missing feature values
print("Number of missing feature values by column: ")
print(train_df.isna().sum())

Number of missing feature values by column: 
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64


In [7]:
# Alternatively we can check the total number of missing feature values in the dataset this way. 
print("Number of missing feature values by column in test data:", test_df.isna().sum().sum())

Number of missing feature values by column in test data: 0


## Feature Scaling

Problem: Typically since feature column values are combined (often by adding them together) to create the final classification, the ML model will perform better if the features are on the same scale. 

Possible Feature Scaling Approaches: 
1. Min-Max Scaling - Scales the data to a fixed range between two values (typically 0 and 1). Most useful for neural networks.
2. Standardization (Z-score Normalization) - Scales the data so that the mean is 0 and the standard deviation is 1. Most useful for algorithms that assume a normal distribution of data, such as SVMs and logistic regression.
3. Robust Scaling - Scaling based on median and IQR. Most useful for handling significant outliers.
4. MaxAbsScaler - Scales each feature based on its maximum absolute value. Useful for sparse data. 

Chosen Approach: Standardization - Since XGBoost's decision tree classification uses splitting which occurs within a column, different column values do not interact with each other and therefore scaling the features is not necessary. However, since we are using Logistic Regression as our baseline model and features do interact in training (they add together to determine the model's class prediction) we will need to do feature scaling (you don't want one column to be in the millions and the other to be in the single digits). Since we are doing feature scaling specifically for our logistic regression model we will use the Standardization approach. 

In [8]:
from sklearn.preprocessing import StandardScaler 

numerical_cols = train_df.columns.difference(categorical_cols)

std_scaler = StandardScaler() 
std_scaler.fit(train_df[numerical_cols])

train_df[numerical_cols] = std_scaler.transform(train_df[numerical_cols])
test_df[numerical_cols] = std_scaler.transform(test_df[numerical_cols])
                      
# Confirm the transformation was successful by seeing if the mean = 0 and std = 1 for numerical columns
test_df.describe()

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000,5271.000000
mean,0.007790,0.033864,0.001549,0.009851,-0.029617,0.010526,-0.002895,-0.020847,-0.001029,-0.013781,0.004353,0.005175
std,0.993309,0.973958,1.010406,1.025961,1.025240,1.021709,1.016664,1.003627,1.001441,1.001418,1.014105,0.987967
min,-1.817984,-2.384728,-0.601797,-1.180148,-4.588553,-1.052815,-0.802543,-1.808946,-1.567576,-2.323678,-2.084550,-2.345776
25%,-0.727654,-0.718300,-0.496885,-0.524971,-0.572940,-0.591648,-0.522026,-0.811772,-0.596648,-0.642910,-0.805263,-1.191649
50%,-0.135421,0.117632,-0.391973,-0.263923,0.062625,-0.249068,-0.354452,-0.156896,-0.215892,-0.071221,-0.037691,-0.037522
75%,0.604869,0.627996,0.106359,0.201867,0.640411,0.238452,0.011428,0.478508,0.431393,0.591939,0.474024,1.116605
max,2.526884,2.792831,6.663359,7.291089,3.153780,6.642081,6.327728,7.023169,9.188781,3.404652,5.847030,1.116605


## Handling Categorical Attributes/Columns 
Problem: We use the DataFrame info method to find that there are six categorical columns: drug, sex, ascites, hepatomegaly, spiders, edema which we need to convert from text into numerical values since machine learning models cannot handle text data naturally, they can only handle numbers.

Possible Approaches: The main approaches for categorical attribute handling are 
1. Ordinal Encoding - Useful when the categories correspond to an ascending or descending order. 
2. One-Hot Encoding - For each categorical column, convert it into multiple columns, one for each possible category. This is used when the categories do not have an obvious logical order. 
3. Numerical Feature Replacement (Advanced) - In cases where the number of categories is cery large (hundreds or thousands) one should consider replacing the categorical columns with a numerical column that converts each category into some number. For example, one could convert a country code into the country's population. 
4. Embedding Replacement (Advanced) - Alternatively, one can replace categories with embeddings, which are low dimensional vectors that represent the category. 

Chosen Approach : One-Hot Encoding - In this case, we use a one-hot encoding since none of the categories seem to have an order to the classes (ruling out ordinal encoding) and the number of categories for each column is low (under 10 for all categorical columns) which rules out needing advanced methods. 

In [9]:
# Confirm that the number of categories in the categorical columns is manageable (< 100)
unique_values_per_column = train_df.nunique()
print(unique_values_per_column)

N_Days           461
Drug               2
Age              391
Sex                2
Ascites            2
Hepatomegaly       2
Spiders            2
Edema              3
Bilirubin        111
Cholesterol      226
Albumin          160
Copper           171
Alk_Phos         364
SGOT             206
Tryglicerides    154
Platelets        227
Prothrombin       49
Stage              4
Status             3
dtype: int64


In [10]:
# Convert the categorical columns into one-hot encodings
status = train_df['Status']
train_df_dummies = pd.get_dummies(train_df.drop('Status', axis=1))
train_df = pd.concat([train_df_dummies, status], axis=1)
test_df = pd.get_dummies(test_df)

# Confirm the transformation was successful
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   N_Days                5271 non-null   float64
 1   Age                   5271 non-null   float64
 2   Bilirubin             5271 non-null   float64
 3   Cholesterol           5271 non-null   float64
 4   Albumin               5271 non-null   float64
 5   Copper                5271 non-null   float64
 6   Alk_Phos              5271 non-null   float64
 7   SGOT                  5271 non-null   float64
 8   Tryglicerides         5271 non-null   float64
 9   Platelets             5271 non-null   float64
 10  Prothrombin           5271 non-null   float64
 11  Stage                 5271 non-null   float64
 12  Drug_D-penicillamine  5271 non-null   bool   
 13  Drug_Placebo          5271 non-null   bool   
 14  Sex_F                 5271 non-null   bool   
 15  Sex_M                

## Train/Test Split

Problem: We need to split the dataset into a training dataset and a testing/tuning dataset.

Possible Approaches
1. K-Fold Cross-Validation - In this approach the dataset is divided into k equal-sized subsets. Then we train the model k times, each time using a different subset as the validation set and the other k-1 subsets as the training set. Finally we use the average score among all k models as the final score. This approach is commonly used since it essentially utilizes  more of the training data for validation (90% vs. 80%). Typical values range are k=5 or k=10. 
2. Train/Validation/Test Split - In this approach we split the dataset into three distinct sets: a training set for training the model, a validation set for tuning the hyperparameters and a test set for evaluating the final model. By separating the test set and validation set we reduce / avoid overfitting to the test set. A typical split varies but could be 80/10/10 or 70/15/15.

Chosen Approach : Initially we used the train/validation/test approach since it is simpler to implement. However, we decided that implementing that impementing k-fold cross validation was not too much harder and would utilize more of the data for training / tuning so eventually we settled on this approach with k=10. 

In [11]:
from sklearn.model_selection import StratifiedKFold

# Set parameters for k-folds cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True)

from sklearn.model_selection import train_test_split

# Split the training set into a training and test set 
X = train_df.drop("Status", axis=1)
y = train_df["Status"]

X_train, X_val, y_train, y_val = train_test_split(X, y,  test_size = 0.2)

# Logistic Regression Baseline Model

In [12]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import log_loss

# Train the model using sci-kit learn's Logistic Regression model 
model = LogisticRegression(max_iter = 1000)
# The fit method learns the parameters (weights) for the model
model.fit(X_train, y_train)
# Predict the probability of each class for the validation set and the test set 
y_val_pred_lr = model.predict_proba(X_val)
y_pred_lr = model.predict_proba(test_df)
print(y_val_pred_lr, "\n")

# Evaluate the model predictions by calculating log loss for the validation set
loss = log_loss(y_val, y_val_pred_lr)
print(f'Validation Set Log Loss: {loss}')

[[9.02992366e-01 1.05900228e-02 8.64176112e-02]
 [4.49692868e-01 7.06604274e-02 4.79646705e-01]
 [7.00271409e-01 5.75837394e-02 2.42144852e-01]
 ...
 [7.61899198e-01 4.46811067e-02 1.93419696e-01]
 [5.34713391e-04 3.44625616e-03 9.96019030e-01]
 [2.19403140e-01 2.81589678e-02 7.52437892e-01]] 

Validation Set Log Loss: 0.5345424812587198


# Random Forest Model 

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Train the model using sci-kit learn's Random Forest model 
rf_classifier = RandomForestClassifier(n_estimators=600, max_depth=30, min_samples_split=10) 
# The fit method learns the parameters (weights) for the model
rf_classifier.fit(X_train, y_train) 
# Predict the probability of each class for the validation set and the test set (Kaggle's unlabeled dataset)
y_val_pred_rf = rf_classifier.predict_proba(X_val)
y_pred_rf = rf_classifier.predict_proba(test_df)
print(y_val_pred_rf, "\n")

# Evaluate the model predictions by calculating log loss for the validation set
loss = log_loss(y_val, y_val_pred_rf)
print(f'Validation Set Log Loss: {loss}')

[[9.57301210e-01 1.27254690e-03 4.14262427e-02]
 [6.29228569e-01 4.74308603e-02 3.23340570e-01]
 [6.61118158e-01 3.40294719e-02 3.04852370e-01]
 ...
 [8.28900117e-01 2.54099201e-02 1.45689963e-01]
 [5.50251138e-03 5.27777778e-04 9.93969711e-01]
 [4.03747756e-01 4.42960697e-02 5.51956174e-01]] 

Validation Set Log Loss: 0.45299829287372545


# XGBoost Model

In [14]:
from xgboost import XGBClassifier 

# Train the model using xgboost's XGBClassifier model 
xgb_model = XGBClassifier(n_estimators=607, max_depth=6, learning_rate=0.0419, colsample_bytree=0.168, min_child_weight=17, subsample=0.7) 
# The fit method learns the parameters (weights) for the model
xgb_model.fit(X_train, y_train)
# Predict the probability of each class for the validation set and the test set (Kaggle's unlabeled dataset)
y_val_pred_xgb = xgb_model.predict_proba(X_val)
y_pred_xgb = xgb_model.predict_proba(test_df)
print(y_val_pred_xgb, "\n")

# Evaluate the model predictions by calculating log loss for the validation set
loss = log_loss(y_val, y_val_pred_xgb)
print(f'Validation Set Log Loss: {loss}')

[[0.95512545 0.00346334 0.04141126]
 [0.7542091  0.02001405 0.22577679]
 [0.6024575  0.06493251 0.33260995]
 ...
 [0.89321953 0.00345223 0.10332823]
 [0.00363269 0.00197448 0.99439275]
 [0.43488002 0.02149551 0.54362446]] 

Validation Set Log Loss: 0.4122643698079633


In [15]:
# Define a function for doing GridSearch/RandomizedSearchCV for hyperparameter tuning 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def grid_search(X, y, param_grid, random=False): 
    xgb = XGBClassifier() 
    
    if random: 
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter = 20, n_jobs=-1)
        
    else :
        grid = GridSearchCV(xgb, param_grid=param_grid, cv=kfold, scoring='neg_log_loss', n_jobs=-1)
        
    grid.fit(X,y) 
    
    best_params = grid.best_params_
    print("Best params:", best_params)
    
    best_score = grid.best_score_
    print("Training score: {:.3f}".format(best_score))

In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Run this cell once for each hyperparameter, finding the best value for that hyperparameter
grid_search(X_train, y_train, param_grid = {
    'n_estimators': [607],
    'learning_rate': [0.0419],
    'max_depth': [6],
    'min_child_weight': [17]})

Best params: {'learning_rate': 0.0419, 'max_depth': 6, 'min_child_weight': 17, 'n_estimators': 607}
Training score: -0.470


## LGBM Classifier

In [17]:
import lightgbm as lgb

# Define your parameters
lgb_params = {
    'max_depth': 15,
    'min_child_samples': 13,
    'learning_rate': 0.05285597081335651,
    'n_estimators': 294,
    'min_child_weight': 5,
    'colsample_bytree': 0.10012816493265511,
    'reg_alpha': 0.8767668608061822,
    'reg_lambda': 0.8705834466355764
}

# Create the LGBMClassifier with the specified parameters
lgbm_model = lgb.LGBMClassifier(**lgb_params)

# Now you can fit this classifier to your data
lgbm_model.fit(X_train, y_train)

# And make predictions
y_val_pred_lgbm = lgbm_model.predict_proba(X_val)
y_pred_lgbm = lgbm_model.predict_proba(test_df)

# Evaluate the model predictions by calculating log loss for the validation set
loss = log_loss(y_val, y_val_pred_lgbm)
print(f'Validation Set Log Loss: {loss}')

Validation Set Log Loss: 0.4111899230141859


# Ensemble Model

In [18]:
from sklearn.ensemble import VotingClassifier

# Train an Ensemble model using a combination of the Random Forest, XGBoost, and LGBM Classifiers

ensemble_model = VotingClassifier(
    estimators=[
        ('lgb', lgbm_model),
        ('xgb', xgb_model),
        ('rf', rf_classifier),
    ],
    voting='soft'
)

ensemble_model.fit(X_train, y_train)

# Make predictions using the ensemble
y_val_pred_ensemble = ensemble_model.predict_proba(X_val)
y_pred_ensemble = ensemble_model.predict_proba(test_df)

# Evaluate the model predictions by calculating log loss for the validation set
loss = log_loss(y_val, y_val_pred_ensemble)
print(f'Validation Set Log Loss: {loss}')

Validation Set Log Loss: 0.41335557452604627


# Kaggle Submission Processing
1. Create a submission dataframe from the model's predictions 
2. Concatenate the data id column values to adhere to submission formatting requirements
3. Convert the submission dataframe into a csv file for submission 
4. Now in order to submit to Kaggle, save the notebook and navigate to the Submissions page for this competition and click 'Submit Prediction' in the top-right corner -> Notebook -> Submit. 

In [19]:
# Modify the probability predictions into the submission format 
submission_df = pd.DataFrame(y_pred_ensemble, columns=['Status_C', 'Status_CL', 'Status_D'])
final_submission_df = pd.concat([test_id_df, submission_df], axis=1)
final_submission_df.head(10)

# Create a submission.csv file that Kaggle will automatically evaluate for submission
final_submission_df.to_csv('submission.csv', index = False)